In [ ]:
!pip install transformers
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e2387b3687830b59d137682d48e2c1415c2cafc1af9aa321284c9d0c8ef2d811
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import xml.etree.ElementTree as ET
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Dataset class for reading and processing XML files
class CdsDataset(Dataset):
    def __init__(self, xml_file_paths):
        self.data = []
        for xml_file_path in xml_file_paths:
            tree = ET.parse(xml_file_path)
            root = tree.getroot()
            for topic in root.findall("topic"):
                description = topic.find("description").text
                summary = topic.find("summary").text
                self.data.append((description, summary))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        description, summary = self.data[index]
        return description, summary

In [ ]:
# Initialize BART tokenizer and model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

In [ ]:
# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [ ]:
# Set training and testing dataset paths
train_dataset_paths = ["/content/drive/MyDrive/Elsevier /LG LLM/topics2014.xml",
                       "/content/drive/MyDrive/Elsevier /LG LLM/topics2015A.xml"
                       ]
test_dataset_path = "/content/drive/MyDrive/Elsevier /LG LLM/topics2016.xml"

# Load training and testing datasets
train_dataset = CdsDataset(train_dataset_paths)
test_dataset = CdsDataset([test_dataset_path])

# Define batch size and number of training epochs
#batch_size = 4
batch_size = 30            # experiment
num_epochs = 1           #try low

# Define optimizer and learning rate
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Initialize BLEU and ROUGE scorers
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

# Training loop
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
len(train_dataset)

60

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
#checking what is inside train_dataloader

import torch

# Iterate over the train_dataloader to access the elements
for batch_idx, data in enumerate(train_dataloader):
    # `data` contains the elements of each batch
    inputs, labels = data

    # You can now inspect the inputs and labels
    print(f"Batch {batch_idx}:")
    print("Inputs:", inputs)
    print("Labels:", labels)

    # You can also access individual elements within the inputs and labels tensors
    # For example, to access the first element of the inputs tensor:
    first_input = inputs[0]

    # Do further processing or analysis as needed


Batch 0:
Inputs: ('A 27-year-old woman at 11 weeks gestation in her second pregnancy is found to have a hemoglobin (Hb) of 9.0 g/dL, white blood cell count 6.3 x 109/L, platelet count 119 x 109/L, mean corpuscular volume 109 fL. Further investigations reveal mild iron deficiency. She already receives iron supplementation. The obstetrician repeats the complete blood cell count 2 weeks later. The Hb is 8.9 g/dL, WBC count 7.1 x 109/L, and platelets 108 x 109/L. She describes difficulty swallowing. A reticulocyte count is performed and found elevated at 180 x 109/L. The obstetrician requests a hematology consult. The following additional results were found: Negative DAT, normal clotting screen, elevated LDH (2000 IU/L), normal urea and electrolytes, normal alanine aminotransferase (ALT), anisocytosis, poikilocytosis, no fragments, no agglutination, polychromasia and presence of hemosiderin in the urine.', 'An 18-year-old male returning from a recent vacation in Asia presents to the ER wit

In [ ]:
len(train_dataloader)

2

In [ ]:
# rough/debugging

for descriptions, summaries in train_dataloader:
        descriptions = list(descriptions)
        print(descriptions)
        summaries = list(summaries)
        print(summaries)

['An 89-year-old man was brought to the emergency department by his wife and son after six months of progressive changes in cognition and personality. He began to have poor memory, difficulty expressing himself, and exhibited unusual behaviors, such as pouring milk onto the table and undressing immediately after getting dressed. He is unable to dress, bathe, use the toilet, or walk independently. On examination the patient\'s temperature was 36.5°C (97.7°F), the heart rate 61 bpm in an irregular rhythm, the blood pressure 144/78 mm Hg, and the respiratory rate 18 bpm. The patient was alert and oriented to self and city but not year. He frequently interrupted the examiner. He repeatedly reached out to grab things in front of him, including the examiner\'s tie and face. He could not spell the word "world" backward, could not follow commands involving multiple steps and was unable to perform simple calculations. His speech was fluent, but he often used similar-sounding word substitutions.

In [ ]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for descriptions, summaries in train_dataloader:
        descriptions = list(descriptions)
        summaries = list(summaries)

        # Tokenize input descriptions and target summaries
        input_encodings = tokenizer.batch_encode_plus(descriptions, padding=True, truncation=True, max_length=512, return_tensors="pt")   #verify
        print(input_encodings)
        target_encodings = tokenizer.batch_encode_plus(summaries, padding=True, truncation=True, max_length=128, return_tensors="pt")     #verify
        print(target_encodings)

        input_ids = input_encodings["input_ids"].to(device)
        attention_mask = input_encodings["attention_mask"].to(device)
        target_ids = target_encodings["input_ids"].to(device)
        target_attention_mask = target_encodings["attention_mask"].to(device)

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=target_ids, decoder_attention_mask=target_attention_mask, labels=target_ids) #verify
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {avg_loss}")
    print("total loss",total_loss)


{'input_ids': tensor([[   0,  250,  971,  ...,    1,    1,    1],
        [   0,  250, 4772,  ...,    1,    1,    1],
        [   0,  250,  291,  ...,    1,    1,    1],
        ...,
        [   0,  250,  333,  ...,    1,    1,    1],
        [   0,  250,  974,  ...,    1,    1,    1],
        [   0,  250,  733,  ...,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
{'input_ids': tensor([[    0,  4688, 24409,  ...,     1,     1,     1],
        [    0,  4419,    12,  ...,     1,     1,     1],
        [    0,   250,   820,  ...,     1,     1,     1],
        ...,
        [    0, 25976,  5796,  ...,     1,     1,     1],
        [    0,   250,   664,  ...,     1,     1,     1],
        [    0,  2146,    12,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 

In [ ]:
#checking test_dataloader/debugging
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(len(test_dataset))

import torch

# Iterate over the test_dataloader to access the elements
for batch_idx, data in enumerate(test_dataloader):
    # `data` contains the elements of each batch
    inputs, labels = data

    # You can now inspect the inputs and labels
    print(f"Batch {batch_idx}:")
    print("Inputs:", inputs)
    print("Labels:", labels)

    # You can also access individual elements within the inputs and labels tensors
    # For example, to access the first element of the inputs tensor:
    first_input = inputs[0]

    # Do further processing or analysis as needed


30
Batch 0:
Inputs: ('78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.', 'An elderly female with past medical history of hypertension, severe aortic stenosis, hyperlipidemia, and right hip arthroplasty. Presents after feeling a snap of her right leg and falling to the ground. No head trauma or loss of consciousness.', 'A 75F with a PMHx significant for severe PVD, CAD, DM, and CKD presented after being found down unresponsive at home. She was found to be hypoglycemic to 29 with hypotension and bradycardia. Her hypotension and confusion improved with hydration. She had a positive UA which eventually grew klebsiella. She had temp 96.3, respiratory rate 22, BP 102/26, a leukocytosis to 18 and a creatinine of 6 (baseline 2). Pt has blood cultures positive for group A str

In [ ]:
for descriptions, summaries in test_dataloader:
        descriptions = list(descriptions)
        print(descriptions)
        summaries = list(summaries)
        print(summaries)

['78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.', 'An elderly female with past medical history of hypertension, severe aortic stenosis, hyperlipidemia, and right hip arthroplasty. Presents after feeling a snap of her right leg and falling to the ground. No head trauma or loss of consciousness.', 'A 75F with a PMHx significant for severe PVD, CAD, DM, and CKD presented after being found down unresponsive at home. She was found to be hypoglycemic to 29 with hypotension and bradycardia. Her hypotension and confusion improved with hydration. She had a positive UA which eventually grew klebsiella. She had temp 96.3, respiratory rate 22, BP 102/26, a leukocytosis to 18 and a creatinine of 6 (baseline 2). Pt has blood cultures positive for group A streptococcus. On the d

In [ ]:
total_bleu_score = 0
total_rouge1_score = 0
total_rouge2_score = 0
total_rougeL_score = 0
total_samples = 0

In [ ]:
generated_summaries =[]

In [ ]:
# rough/evaluation debug

model.eval()
for descriptions, summaries in test_dataloader:
    descriptions = list(descriptions)
    summaries = list(summaries)

    # Tokenize input descriptions and target summaries
    input_encodings = tokenizer.batch_encode_plus(descriptions, padding=True, truncation=True, max_length=512, return_tensors="pt")
    target_encodings = tokenizer.batch_encode_plus(summaries, padding=True, truncation=True, max_length=128, return_tensors="pt")

    input_ids = input_encodings["input_ids"].to(device)
    attention_mask = input_encodings["attention_mask"].to(device)
    target_ids = target_encodings["input_ids"].to(device)
    target_attention_mask = target_encodings["attention_mask"].to(device)

    # Generate summaries
    with torch.no_grad():
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128, num_beams=4, early_stopping=True)


    #generated_summaries = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    for generated_ids_batch in generated_ids:
      decoded_summaries = tokenizer.batch_decode(generated_ids_batch, skip_special_tokens=True)
      generated_summaries.extend(decoded_summaries)




In [ ]:
generated_summaries

['</s>',
 '<s>',
 '78',
 '78',
 '78',
 ' 78',
 '78',
 ' 78',
 ' 78',
 ' 78',
 ' 79',
 ' 79',
 ' 79',
 ' 78',
 ' 78',
 ' 76',
 ' 76',
 ' 76',
 ' 79',
 ' 79',
 ' 76',
 ' 79',
 ' 76',
 ' 76',
 ' 77',
 ' 76',
 ' 76',
 ' 72',
 ' 72',
 ' 72',
 ' 73',
 ' 73',
 ' 73',
 ' 72',
 ' 72',
 ' 69',
 ' 69',
 ' 69',
 ' 67',
 ' 67',
 ' 67',
 ' 66',
 ' 66',
 ' 66',
 ' 67',
 ' 67',
 ' 68',
 ' 67',
 ' 67',
 ' 69',
 ' 69',
 ' 66',
 ' 67',
 ' 66',
 ' 67',
 ' 69',
 ' 66',
 ' 66',
 ' 69',
 ' 67',
 ' 69',
 ' 67',
 ' 66',
 ' 69',
 ' 69',
 ' 68',
 ' 67',
 ' 69',
 ' 68',
 ' 69',
 ' 67',
 ' 68',
 ' 69',
 ' 69',
 ' 70',
 ' 67',
 ' 67',
 ' 71',
 ' 67',
 ' 67',
 ' 65',
 ' 67',
 ' 67',
 ' 70',
 ' 67',
 ' 69',
 ' 70',
 ' 70',
 ' 70',
 ' 65',
 ' 67',
 ' 68',
 ' 68',
 ' 67',
 ' 66',
 ' 68',
 ' 67',
 ' 68',
 ' 66',
 ' 67',
 ' 70',
 ' 70',
 ' 67',
 ' 68',
 ' 65',
 ' 67',
 ' 69',
 ' 65',
 ' 67',
 ' 66',
 ' 65',
 ' 67',
 ' 70',
 ' 65',
 ' 65',
 ' 65',
 ' 70',
 ' 70',
 ' 60',
 ' 65',
 ' 65',
 ' 66',
 ' 66',
 ' 65',
 ' 65',
 ' 

In [ ]:
# Evaluation on the test dataset
model.eval()
for descriptions, summaries in test_dataloader:
    descriptions = list(descriptions)
    summaries = list(summaries)

    # Tokenize input descriptions and target summaries
    input_encodings = tokenizer.batch_encode_plus(descriptions, padding=True, truncation=True, max_length=512, return_tensors="pt")
    target_encodings = tokenizer.batch_encode_plus(summaries, padding=True, truncation=True, max_length=128, return_tensors="pt")

    input_ids = input_encodings["input_ids"].to(device)
    attention_mask = input_encodings["attention_mask"].to(device)
    target_ids = target_encodings["input_ids"].to(device)
    target_attention_mask = target_encodings["attention_mask"].to(device)

    # Generate summaries
    with torch.no_grad():
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128, num_beams=4, early_stopping=True)

    # Calculate BLEU and ROUGE scores
    generated_summaries = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    bleu_score = corpus_bleu([[ref] for ref in summaries], generated_summaries)
    rouge_scores = rouge_scorer.score(generated_summaries, summaries)

    # Accumulate scores and sample counts
    total_bleu_score += bleu_score.score
    total_rouge1_score += rouge_scores["rouge1"].fmeasure
    total_rouge2_score += rouge_scores["rouge2"].fmeasure
    total_rougeL_score += rouge_scores["rougeL"].fmeasure
    total_samples += len(descriptions)

# Calculate average scores
avg_bleu_score = total_bleu_score / total_samples
avg_rouge1_score = total_rouge1_score / total_samples
avg_rouge2_score = total_rouge2_score / total_samples
avg_rougeL_score = total_rougeL_score / total_samples

# Print evaluation results
print("Evaluation Results:")
print(f"BLEU Score: {avg_bleu_score:.4f}")
print(f"ROUGE-1 Score: {avg_rouge1_score:.4f}")
print(f"ROUGE-2 Score: {avg_rouge2_score:.4f}")
print(f"ROUGE-L Score: {avg_rougeL_score:.4f}")


AttributeError: ignored

In [ ]:
# rough
# Evaluation on the test dataset
model.eval()
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
total_bleu_score = 0
total_rouge1_score = 0
total_rouge2_score = 0
total_rougeL_score = 0
total_samples = 0

for descriptions, summaries in test_dataloader:
    descriptions = list(descriptions)
    summaries = list(summaries)

    # Tokenize input descriptions and target summaries
    input_encodings = tokenizer.batch_encode_plus(descriptions, padding=True, truncation=True, max_length=512, return_tensors="pt")
    target_encodings = tokenizer.batch_encode_plus(summaries, padding=True, truncation=True, max_length=128, return_tensors="pt")

    input_ids = input_encodings["input_ids"].to(device)
    attention_mask = input_encodings["attention_mask"].to(device)
    target_ids = target_encodings["input_ids"].to(device)
    target_attention_mask = target_encodings["attention_mask"].to(device)

    # Generate summaries
    with torch.no_grad():
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128, num_beams=4, early_stopping=True)

    # Decode generated summaries
    generated_summaries = tokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    # Convert summaries to strings
    summaries = [summary.decode("utf-8") for summary in summaries]

    # Calculate BLEU and ROUGE scores
    bleu_score = corpus_bleu([[ref] for ref in summaries], generated_summaries)
    rouge_scores = rouge_scorer.score(generated_summaries, summaries)

    # Accumulate scores and sample counts
    total_bleu_score += bleu_score.score
    total_rouge1_score += rouge_scores["rouge1"].fmeasure
    total_rouge2_score += rouge_scores["rouge2"].fmeasure
    total_rougeL_score += rouge_scores["rougeL"].fmeasure
    total_samples += len(descriptions)

# Calculate average scores
avg_bleu_score = total_bleu_score / total_samples
avg_rouge1_score = total_rouge1_score / total_samples
avg_rouge2_score = total_rouge2_score / total_samples
avg_rougeL_score = total_rougeL_score / total_samples

# Print evaluation results
print("Evaluation Results:")
print(f"BLEU Score: {avg_bleu_score:.4f}")
print(f"ROUGE-1 Score: {avg_rouge1_score:.4f}")
print(f"ROUGE-2 Score: {avg_rouge2_score:.4f}")
print(f"ROUGE-L Score: {avg_rougeL_score:.4f}")


AttributeError: ignored

In [ ]:
#rough2
# rough
# Evaluation on the test dataset
model.eval()
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
total_bleu_score = 0
total_rouge1_score = 0
total_rouge2_score = 0
total_rougeL_score = 0
total_samples = 0

for descriptions, summaries in test_dataloader:
    descriptions = list(descriptions)
    summaries = list(summaries)

    # Tokenize input descriptions and target summaries
    input_encodings = tokenizer.batch_encode_plus(descriptions, padding=True, truncation=True, max_length=512, return_tensors="pt")
    target_encodings = tokenizer.batch_encode_plus(summaries, padding=True, truncation=True, max_length=128, return_tensors="pt")

    input_ids = input_encodings["input_ids"].to(device)
    attention_mask = input_encodings["attention_mask"].to(device)
    target_ids = target_encodings["input_ids"].to(device)
    target_attention_mask = target_encodings["attention_mask"].to(device)

    # Generate summaries
    with torch.no_grad():
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128, num_beams=4, early_stopping=True)

    # Decode generated summaries
    generated_summaries = tokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    # Convert lists to strings
    generated_summaries = [' '.join(summary) for summary in generated_summaries]
    summaries = [' '.join(summary) for summary in summaries]

    # Convert strings to lowercase
    generated_summaries = [summary.lower() for summary in generated_summaries]
    summaries = [summary.lower() for summary in summaries]

    bleu_score = corpus_bleu([[ref] for ref in summaries], generated_summaries)
    rouge_scores = rouge_scorer.score(generated_summaries, summaries)


    # Calculate BLEU and ROUGE scores
    #bleu_score = corpus_bleu([[ref] for ref in summaries], generated_summaries)
    #rouge_scores = rouge_scorer.score(generated_summaries, summaries)

    # Accumulate scores and sample counts
    total_bleu_score += bleu_score.score
    total_rouge1_score += rouge_scores["rouge1"].fmeasure
    total_rouge2_score += rouge_scores["rouge2"].fmeasure
    total_rougeL_score += rouge_scores["rougeL"].fmeasure
    total_samples += len(descriptions)

# Calculate average scores
avg_bleu_score = total_bleu_score / total_samples
avg_rouge1_score = total_rouge1_score / total_samples
avg_rouge2_score = total_rouge2_score / total_samples
avg_rougeL_score = total_rougeL_score / total_samples

# Print evaluation results
print("Evaluation Results:")
print(f"BLEU Score: {avg_bleu_score:.4f}")
print(f"ROUGE-1 Score: {avg_rouge1_score:.4f}")
print(f"ROUGE-2 Score: {avg_rouge2_score:.4f}")
print(f"ROUGE-L Score: {avg_rougeL_score:.4f}")


AttributeError: ignored

In [ ]:
# Path to the XML file
xml_file_path = "/content/drive/MyDrive/Elsevier /LG LLM/topics2016.xml"

# Parse the XML file
tree = ET.parse(xml_file_path)
root = tree.getroot()

In [ ]:
clinical_notes = []
reference_summaries = []
for topic in root.findall("topic"):
    description = topic.find("description").text
    clinical_notes.append(description)
    summary = topic.find("summary").text
    reference_summaries.append(summary)

In [ ]:
print(clinical_notes)
print(reference_summaries)

['78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.', 'An elderly female with past medical history of hypertension, severe aortic stenosis, hyperlipidemia, and right hip arthroplasty. Presents after feeling a snap of her right leg and falling to the ground. No head trauma or loss of consciousness.', 'A 75F with a PMHx significant for severe PVD, CAD, DM, and CKD presented after being found down unresponsive at home. She was found to be hypoglycemic to 29 with hypotension and bradycardia. Her hypotension and confusion improved with hydration. She had a positive UA which eventually grew klebsiella. She had temp 96.3, respiratory rate 22, BP 102/26, a leukocytosis to 18 and a creatinine of 6 (baseline 2). Pt has blood cultures positive for group A streptococcus. On the d

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
# Initialize BLEU and ROUGE scorers
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

# Summarize clinical notes and evaluate
for i, note in enumerate(clinical_notes):
    inputs = tokenizer.encode(note, return_tensors="pt", max_length=1024, truncation=True).to(device)
    summary_ids = model.generate(inputs, num_beams=4, max_length=100, early_stopping=True)
    summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)

    # Evaluate BLEU score
    bleu_score = corpus_bleu([[reference_summaries[i].split()]], [summary.split()])
    bleu_scores.append(bleu_score)

    # Evaluate ROUGE scores
    reference = reference_summaries[i]
    hypothesis = summary
    rouge_scores = rouge_scorer.score(reference, hypothesis)

    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

    print("Clinical Note:")
    print(note)
    print("\nReference Summary:")
    print(reference_summaries[i])
    print("\nGenerated Summary:")
    print(summary)
    print("\nBLEU Score:", bleu_score)
    print("ROUGE Scores:", rouge_scores)
    print("--------------------------------------------------")

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Clinical Note:
78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.

Reference Summary:
A 78 year old male presents with frequent stools and melena.

Generated Summary:
78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.

BLEU Score: 9.157231061812019e-232
ROUGE Scores: {'rouge1': Score(precision=0.10638297872340426, recall=0.45454545454545453, fmeasure=0.17241379310344826), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0851063829787234, recall=0.36363636363636365, fmeasure=0.13793103448275862)}
--------------------

In [ ]:
# Calculate and print average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU Score:", avg_bleu_score)

# Calculate and print average ROUGE scores
avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)
print("Average ROUGE-1 Score:", avg_rouge1_score)
print("Average ROUGE-2 Score:", avg_rouge2_score)
print("Average ROUGE-L Score:", avg_rougeL_score)

Average BLEU Score: 0.1495721018813023
Average ROUGE-1 Score: 0.372939928051619
Average ROUGE-2 Score: 0.25427201580633274
Average ROUGE-L Score: 0.33055007486239435


In [ ]:
# previous experiments

In [ ]:
# without finetuning
bart large
Average BLEU Score: 0.1591846009150742
Average ROUGE-1 Score: 0.3877955304465891
Average ROUGE-2 Score: 0.27000199960889637
Average ROUGE-L Score: 0.35379814397744974

Bart Base
Average BLEU Score: 0.1783641498431823
Average ROUGE-1 Score: 0.405226409066793
Average ROUGE-2 Score: 0.2918551636750237
Average ROUGE-L Score: 0.36829885204737667


In [ ]:
epoch - 1, batch size - 5

Average BLEU Score: 0.016357416463445906
Average ROUGE-1 Score: 0.16578484753628317
Average ROUGE-2 Score: 0.0646883210811589
Average ROUGE-L Score: 0.1460307306653012

In [ ]:
epoch - 3, batch size - 5

Average BLEU Score: 8.483789026792813e-80
Average ROUGE-1 Score: 0.08028617630915373
Average ROUGE-2 Score: 0.0108147053206842
Average ROUGE-L Score: 0.07207405995558597

#definitely gave much better results than the previous one but still bizzare

In [ ]:
epoch - 6, batch size - 5

Average BLEU Score: 2.7064445170523664e-156
Average ROUGE-1 Score: 0.04657414636379524
Average ROUGE-2 Score: 0.002872711227083659
Average ROUGE-L Score: 0.0433134658868792

In [ ]:
epoch - 10, batch size - 5

Average BLEU Score: 8.155030667672079e-157
Average ROUGE-1 Score: 0.03299145432029096
Average ROUGE-2 Score: 0.0005509641873278236
Average ROUGE-L Score: 0.03237983352518392

In [ ]:
epoch - 30, batch size - 3

Average BLEU Score: 5.096135128605618e-232
Average ROUGE-1 Score: 0.03214803536477296
Average ROUGE-2 Score: 0.0
Average ROUGE-L Score: 0.03214803536477296

In [ ]:
# from the above it can be concluded that that when batch size is kept fixed and epoch is minimum, the model is performing best

In [ ]:
# now we experiment with batch size

In [ ]:
epoch - 1, batch size - 8

Average BLEU Score: 0.021185278101561542
Average ROUGE-1 Score: 0.17363254168654105
Average ROUGE-2 Score: 0.07399055628648708
Average ROUGE-L Score: 0.15039634224747148

epoch - 1, batch size - 5

Average BLEU Score: 0.016357416463445906
Average ROUGE-1 Score: 0.16578484753628317
Average ROUGE-2 Score: 0.0646883210811589
Average ROUGE-L Score: 0.1460307306653012

In [ ]:
epoch - 1, batch size - 12

Average BLEU Score: 0.08894860960249253
Average ROUGE-1 Score: 0.3007008417989294
Average ROUGE-2 Score: 0.17446907498274583
Average ROUGE-L Score: 0.2603521276643042

In [ ]:
epoch - 1, batch size - 20

Average BLEU Score: 0.13056122664375716
Average ROUGE-1 Score: 0.347750831485092
Average ROUGE-2 Score: 0.2240762809918197
Average ROUGE-L Score: 0.3037268797739914

In [ ]:
epoch - 1, batch size - 30

Average BLEU Score: 0.1495721018813023
Average ROUGE-1 Score: 0.372939928051619
Average ROUGE-2 Score: 0.25427201580633274
Average ROUGE-L Score: 0.33055007486239435

Bart Base ORIGINAL
Average BLEU Score: 0.1783641498431823
Average ROUGE-1 Score: 0.405226409066793
Average ROUGE-2 Score: 0.2918551636750237
Average ROUGE-L Score: 0.36829885204737667

In [ ]:
epoch - 1, batch size - 60

OutOfMemoryError: CUDA out of memory. Tried to allocate 54.00 MiB (GPU 0; 14.75 GiB total capacity; 13.70 GiB already allocated; 44.81 MiB free; 13.86 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# finetuning the model to any extent is just resulting in very bad results.
# makes me wonder if the model is overfitting miserably due to the lack of data

# things i was thinking to experiment
- change the model. use something else intead of BART
- use a more large dataset and try